## Import Libraries

In [1]:
import cv2
import numpy as np
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

## Defining paths to folders

In [2]:
from google.colab import drive
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/CW_Folder_UG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['Report template.docx', '.DS_Store', 'CW_Dataset', '.ipynb_checkpoints', 'Models', 'test_functions.ipynb']


In [3]:
# Define the path to the image folder and label folder
train_image_folder = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, 'CW_Dataset','CV2023_CW_Dataset','train','images')
train_label_folder = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, 'CW_Dataset','CV2023_CW_Dataset','train','labels')
test_image_folder=os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, 'CW_Dataset','CV2023_CW_Dataset','test','images')
test_label_folder=os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, 'CW_Dataset','CV2023_CW_Dataset','test','labels')

## Loading images and labels


In [4]:
# Load the training images and labels
train_images = []
train_labels = []

for filename in os.listdir(train_image_folder):
    img = cv2.imread(os.path.join(train_image_folder, filename), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64)) # Resize the image to 64x64
    train_images.append(img)
    label_file = os.path.join(train_label_folder, f"{os.path.splitext(filename)[0]}.txt")
    with open(label_file, "r") as f:
        label = int(f.read().strip())
        train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Load the testing images and labels
test_images = []
test_labels = []

for filename in os.listdir(test_image_folder):
    img = cv2.imread(os.path.join(test_image_folder, filename), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64)) # Resize the image to 64x64
    test_images.append(img)
    label_file = os.path.join(test_label_folder, f"{os.path.splitext(filename)[0]}.txt")
    with open(label_file, "r") as f:
        label = int(f.read().strip())
        test_labels.append(label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

## preprocessing the data and defining the model

In [6]:
import tensorflow as tf

import tensorflow as tf

# Convert the training and testing data to tensors
train_images = tf.convert_to_tensor(train_images, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)

test_images = tf.convert_to_tensor(test_images, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# Define and compile the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))



Epoch 1/10
75/75 [==============================] - 11s 131ms/step - loss: 1.3916 - accuracy: 0.7794 - val_loss: 0.4294 - val_accuracy: 0.8559
Epoch 2/10
75/75 [==============================] - 10s 133ms/step - loss: 0.3822 - accuracy: 0.8542 - val_loss: 0.3612 - val_accuracy: 0.8799
Epoch 3/10
75/75 [==============================] - 8s 107ms/step - loss: 0.3158 - accuracy: 0.8855 - val_loss: 0.3928 - val_accuracy: 0.8581
Epoch 4/10
75/75 [==============================] - 10s 137ms/step - loss: 0.2587 - accuracy: 0.9060 - val_loss: 0.3098 - val_accuracy: 0.8886
Epoch 5/10
75/75 [==============================] - 9s 123ms/step - loss: 0.2279 - accuracy: 0.9198 - val_loss: 0.3427 - val_accuracy: 0.8843
Epoch 6/10
75/75 [==============================] - 9s 124ms/step - loss: 0.1900 - accuracy: 0.9357 - val_loss: 0.3698 - val_accuracy: 0.8865
Epoch 7/10
75/75 [==============================] - 10s 137ms/step - loss: 0.1921 - accuracy: 0.9353 - val_loss: 0.3728 - val_accuracy: 0.8843
Ep

## Save the model using joblib


In [15]:
import joblib
joblib.dump(model, 'drive/MyDrive/Colab Notebooks/CW_Folder_UG/Models/CNN.joblib')

['drive/MyDrive/Colab Notebooks/CW_Folder_UG/Models/CNN.joblib']

## Test the Model

In [10]:

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc}")

15/15 - 0s - loss: 0.3732 - accuracy: 0.8908 - 427ms/epoch - 28ms/step
Test accuracy: 0.8908296823501587


## Evaluation of the model

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Get the predicted labels for the test set
pred_labels = np.argmax(model.predict(test_images), axis=-1)

# Generate the confusion matrix
cm = confusion_matrix(test_labels, pred_labels)
print("Confusion Matrix:")
print(cm)

15/15 [==============================] - 1s 53ms/step
Confusion Matrix:
[[ 28  23   0]
 [  8 376   4]
 [  1  14   4]]


In [13]:
# Generate the classification report
class_names = ['Class 0', 'Class 1', 'Class 2']
print("Classification Report:")
print(classification_report(test_labels, pred_labels, target_names=class_names))

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.76      0.55      0.64        51
     Class 1       0.91      0.97      0.94       388
     Class 2       0.50      0.21      0.30        19

    accuracy                           0.89       458
   macro avg       0.72      0.58      0.62       458
weighted avg       0.88      0.89      0.88       458

